### 1. 데이터 크롤링

In [3]:
import os
import sys
import requests
import pandas as pd

# 발급받은 키 설정
client_id = "roedCPfA3OHVsgKPfvdW"     
client_secret = "pXtzOkKdH3" 

def get_naver_shopping(keyword, display_count=10):
    url = "https://openapi.naver.com/v1/search/shop.json"
    
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    
    params = {
        "query": keyword,      # 검색어
        "display": display_count, # 출력 건수
        "sort": "sim"          # 유사도순(sim)
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        items = data['items']
        
        # 데이터프레임으로 변환
        df = pd.DataFrame(items)
        
        # 불필요한 HTML 태그 제거 및 기본 정제
        df['title'] = df['title'].str.replace('<b>', '').str.replace('</b>', '') # '<b>, ''
        
        return df[['title', 'lprice', 'mallName', 'category1', 'category2']]
    else:
        print(f"Error Code: {response.status_code}") # 실패시 200 이외의 상태 코드 출력
        return None


In [4]:
# 2. 실행 테스트 (예: 한돈 삼겹살)
keyword = "한돈 삼겹살 500g"
df = get_naver_shopping(keyword)

if df is not None:
    print(f"'{keyword}' 검색 결과:")
    print(df)
    # df.to_csv("naver_shopping_sample.csv", index=False) # CSV 저장 가능
df.to_csv('../02_데이터 폴더/naver_shopping_sample.csv', index=False)


'한돈 삼겹살 500g' 검색 결과:
                                               title lprice       mallName  \
0  지리산 흑돼지 삼겹살 오겹살 목살 통 앞다리살 항정살 가브리살 갈매기살 사태 근고기...  18900     지리산 오부 흑돼지   
1              국가대표돼지 국내산 냉장 생 삼겹살500g 한돈 목살 구이용 보쌈용   9500         국가대표돼지   
2             한돈 선물세트 목살 삼겹살 보성녹돈 돼지고기 구이용 500g+500g  35000        미소드림쇼핑몰   
3  삼겹살 오겹살 목살 1등급 한돈 500g+500g 돼지갈비 등갈비 찌개 사태 안심 1kg  19500         엄지 농수산   
4                      국내산 냉동 한돈 냉삼 옛날삼겹살 냉동삼겹살 500g  12500           행복한돈   
5                    한돈 돼지전지 미박 앞다리살 냉제육 보쌈 수육용 500g   7500           풍진축산   
6  한푸짐 지리산 흑돼지 버크셔 삼겹살(오겹살) 목살 통 앞다리살 항정살 가브리살 갈매...  18900     지리산흑돼지 한푸짐   
7                  [도드람한돈] 삼겹살 구이용 500g +목살 구이용 500g  29000          도드람한돈   
8                          한돈 대패 삼겹살 국내산 냉동 600g, 2개  19600  육육마켓 66Market   
9     한돈 냉장 돼지고기 삼겹살 오겹살 목살 앞다리살 전지 구이용 찌개용 캠핑 더예쁜한우   5940          더예쁜한우   

  category1 category2  
0        식품       축산물  
1        식품       축산물  
2        식품       축산물  
3        식품       축산물  


In [5]:
df['가격'] = df['lprice'].astype(int)
df = df.sort_values(by='가격')
display(df)  # 상위 5개 상품 출력

,title,lprice,mallName,category1,category2,가격
9,한돈 냉장 돼지고기 삼겹살 오겹살 목살 앞다리살 전지 구이용 찌개용 캠핑 더예쁜한우,5940,더예쁜한우,식품,축산물,5940
5,한돈 돼지전지 미박 앞다리살 냉제육 보쌈 수육용 500g,7500,풍진축산,식품,축산물,7500
1,국가대표돼지 국내산 냉장 생 삼겹살500g 한돈 목살 구이용 보쌈용,9500,국가대표돼지,식품,축산물,9500
4,국내산 냉동 한돈 냉삼 옛날삼겹살 냉동삼겹살 500g,12500,행복한돈,식품,축산물,12500
6,한푸짐 지리산 흑돼지 버크셔 삼겹살(오겹살) 목살 통 앞다리살 항정살 가브리살 갈매...,18900,지리산흑돼지 한푸짐,식품,축산물,18900
0,지리산 흑돼지 삼겹살 오겹살 목살 통 앞다리살 항정살 가브리살 갈매기살 사태 근고기...,18900,지리산 오부 흑돼지,식품,축산물,18900
3,삼겹살 오겹살 목살 1등급 한돈 500g+500g 돼지갈비 등갈비 찌개 사태 안심 1kg,19500,엄지 농수산,식품,축산물,19500
8,"한돈 대패 삼겹살 국내산 냉동 600g, 2개",19600,육육마켓 66Market,식품,축산물,19600
7,[도드람한돈] 삼겹살 구이용 500g +목살 구이용 500g,29000,도드람한돈,식품,축산물,29000
2,한돈 선물세트 목살 삼겹살 보성녹돈 돼지고기 구이용 500g+500g,35000,미소드림쇼핑몰,식품,축산물,35000
